_HDS5210 Programming for Health Data Scientists_

# Week 7 - Reading / Writing files

This week, we're talking about reading and writing files to disk on the Jupyter server.  Note that if you want to use Jupyter to process your own files, all you have to do is do `File -> Open` and then click `Upload`.

For our exercises, we're going to use simple text file formats, but you should take a look at claim data file formats (https://www.ihs.gov/hipaa/835_837/newsletter4/) and HL7 clinical file formats (http://hl7api.sourceforge.net/hapi-testpanel/index.html) to see what kinds of files you may run across.

## 1 - Reading simple text files

For this section, we'll be using a file stored on the server in `/week07/aco_year1.csv`.  If you want to see what the file looks like, you can see a layout on the CMS website: https://data.cms.gov/ACO/Medicare-Shared-Savings-Program-Accountable-Care-O/yuq5-65xt


There's a bash command called `head` that will print the first 10 lines of any file.  So, let's use that to take a peak at what this file looks like on disk.

In [ ]:
%%bash
head /week07/aco_year1.csv 

Now, we'll use Pythong to print out the first line of the file.

In [ ]:
with open('/week07/aco_year1.csv') as aco:
    print(aco.readline())

We can also use a loop in python to print out the first ten lines of the file, like `head`

In [ ]:
with open('/week07/aco_year1.csv') as aco:
    for i in range(10):
        print(aco.readline())

## 2 - CSV Files with quoted field values

This isn't mentioned in the book yet, but Python has a special module for reading and writing CSV files that all for the delimiter to appear inside of a quoted field.  Documentation on this module is here: https://docs.python.org/2/library/csv.html#

In [ ]:
import csv
with open('/week07/aco_year1.csv') as aco:
    reader = csv.reader(aco)
    for row in reader:
        print(row)
        break

In [ ]:
import csv
with open('/week07/aco_year1.csv') as aco:
    reader = csv.reader(aco)
    for row in reader:
        print(row)
        break
        
    assigned_pos = row.index('Total Assigned Beneficiaries')
    print('Total assigned is in column {:d}'.format(assigned_pos+1))
    
    total_assigned = 0
    for row in reader:
        total_assigned += int(row[assigned_pos])
        
    print('Total assigned: {:,d}'.format(total_assigned))
        

In [ ]:
import csv

acos = []
with open('/week07/aco_year1.csv') as aco:
    reader = csv.reader(aco)
    for row in reader:
        print(row)
        break
        
    assigned_pos = row.index('Total Assigned Beneficiaries')
    print('Total assigned is in column {:d}'.format(assigned_pos+1))

    for row in reader:
        acos.append([row[0],int(row[assigned_pos])])

acos.sort()
print(acos[0:10])
        
        

## 3 - Reading multi-line records

Imagine, though, if you had a file whose record structures aren't quite so nice and rectangular, like a database table or spreadsheet.  Image if you had something with multiple lines per record (this file is in /week07/med_list.txt):

```
PATIENT:Boal,Paul
MEDICATION:Ibuprofen,200mg
MEDICATION:Vallium,95mg
END
PATIENT:Westhus,Eric
MEDICATION:Acetominaphen,200mg
MEDICATION:Flintstones Chewable Morphine,100mg
MEDICATION:Zolpidem,10mg
END
```

You might want to read this into a list structure in Python that looks something like this:
```
[['Boal', 'Paul', ['Ibuprofen','Vallium']],
 ['Westhus', 'Eric', ['Acetominaphen','Finstones Chewable Morphine','Zolpidem']]
```

In this case, we'll want to read the file one row at a time and decide what to do based on the contents of each line.

In [8]:
med_list = []

with open('/week07/med_list.txt') as meds:
    patient = []
    for row in meds:
        item = row.strip().split(':')     # We strip the newline and split on :
        if item[0] == 'PATIENT':
            patient = item[1].split(',')  # This just sets patient to be the last name, first name
            patient.append([])            # This adds an empty list on to the end so we can add meds
        elif item[0] == 'MEDICATION':
            med = item[1].split(',')[0]
            patient[2].append(med)        # Add the medication name to the end of our list
        elif item[0] == 'END':
            med_list.append(patient)      # Add that whole entry we have onto our main list
        else:
            pass


print(med_list)

[['Boal', 'Paul', ['Ibuprofen', 'Vallium']], ['Westhus', 'Eric', ['Acetominaphen', 'Flintstones Chewable Morphine', 'Zolpidem']]]


## 4 - Writing to Files

In this last piece, we're going to talk through how to write data into an output file in a structured format.  For instance, looking at the previous example, let's try to put together a pipe-delimited output like this:

```
Boal|Paul|Ibuprofen,Vallium
Westhus|Eric|Acetominaphen,Fintstones Chewable Morphine,Zolpidem
```


In [9]:
with open('med_list.txt','w') as output:
    for items in med_list:
        out = "{:s}|{:s}|{:s}\n".format(items[0],items[1],",".join(items[2]))
        output.write(out)

In [10]:
%%bash
cat med_list.txt

Boal|Paul|Ibuprofen,Vallium
Westhus|Eric|Acetominaphen,Flintstones Chewable Morphine,Zolpidem
